Calibration ignores hcal component and tries to fit ecal with linear and fractional term, E^(3/2) to match with HCal/ECal linear extrapolation data. Two calibration constants are gained - a linear and fractional term for the total ecal with assumed relative factor of 2. Done only in a full fit method 

Backscatter removed, no tail cuts

In [1]:
using LCIO
using Plots
using Glob
using LsqFit
using StatsBase
using Distributions
using StatPlots
gr();

In [2]:
# read the files for calibration - store files in `singleParticles` directory
fileList = readdir(glob"reco_5000a*0phi*", "singleParticles")
phi=0.0
# build the list of input energies
energyString = r"_(\d+)GeV"
# get (match) the energy string from the filename, convert (parse) from string to Int16, build a dictionary from ints to filenames
energyMap = Dict((parse(Int16, match(energyString, fn)[1]), fn) for fn in fileList)
for (energy, filename) in energyMap
    println(energy, " GeV:\t", filename)
end

20 GeV:	singleParticles/reco_5000a_20GeV_0phi.slcio
100 GeV:	singleParticles/reco_5000a_100GeV_0phi.slcio
10 GeV:	singleParticles/reco_5000a_10GeV_0phi.slcio
50 GeV:	singleParticles/reco_5000a_50GeV_0phi.slcio
2 GeV:	singleParticles/reco_5000a_2GeV_0phi.slcio
5 GeV:	singleParticles/reco_5000a_5GeV_0phi.slcio
1 GeV:	singleParticles/reco_5000a_1GeV_0phi.slcio


In [3]:
function getHitsFromFile(filename)
    eCalEnergies = Float64[]
    hCalEnergies = Float64[]
    eCalBackEns = Float64[]
    eCalLengths = Int16[]
    LCIO.open(filename) do reader
        for (idx, event) in enumerate(reader)
            if idx > 10000
                break
            end
            eCalBackEn = 0.0
            hCalEnergy = 0.0
            eCalEnergy = 0.0
            eCalLength = 0
            # sum up the uncalibrated ECalHits
            # this needs to be sorted by layer, so we need a decoder
            EcalBarrelHits = getCollection(event, "ECalBarrelHits")
            decode = CellIDDecoder(EcalBarrelHits)
            for hit in EcalBarrelHits
                ##eliminate backscatter - if angle > 0.2 rad from center at "phi"
                angle=(acos((cos(phi*pi/180.)*getPosition(hit)[1]+sin(phi*pi/180.)*getPosition(hit)[2])/sqrt(getPosition(hit)[1]*getPosition(hit)[1]+getPosition(hit)[2]*getPosition(hit)[2]+getPosition(hit)[3]*getPosition(hit)[3])))
                if getPosition(hit)[1]<0
                    angle=-angle
                end
                if phi*pi/180+0.2>angle>phi*pi/180-0.2
                    # calibrate the hits in the later layers with a higher number, because they are behind thicker tungsten slabs
                    factor = decode(hit)["layer"] < 21 ? 1 : 2
                    if decode(hit)["layer"]>0
                        eCalEnergy += factor*getEnergy(hit)
                        eCalLength += 1
                        if decode(hit)["layer"]>=29
                            eCalBackEn+=getEnergy(hit)
                        end
                    end
                end
            end
            HcalBarrelHits = getCollection(event,"HCalBarrelHits")
            for hhit in HcalBarrelHits
                hCalEnergy += getEnergy(hhit) #sum up all raw HCal deposits
            end
            # fixme: Simple outlier cut
            if eCalLength < 30
                continue
            end
            push!(eCalEnergies, eCalEnergy)
            push!(eCalLengths, eCalLength)
            push!(eCalBackEns, eCalBackEn)
            push!(hCalEnergies,hCalEnergy)
        end
    end
    return eCalEnergies, eCalLengths, eCalBackEns, hCalEnergies
end;

In [4]:
eHits  = Dict{Int16, Vector{Float64}}()
hHits  = Dict{Int16, Vector{Float64}}()
eCount = Dict{Int16, Vector{Int16}}()
eHitsBack  = Dict{Int16, Vector{Float64}}()

for (energy, filename) in energyMap
    println("Processing file for ", energy, " GeV")
    eCal, nEhits, eCalBack, hCal = getHitsFromFile(filename)
    eHits[energy] = eCal
    hHits[energy] = hCal
    eCount[energy] = nEhits
    eHitsBack[energy] = eCalBack
end


Processing file for 20 GeV
Processing file for 100 GeV
Processing file for 10 GeV
Processing file for 50 GeV
Processing file for 2 GeV
Processing file for 5 GeV
Processing file for 1 GeV


In [5]:
sigmasUncalib=Vector{Float64}()
meansUncalib=Vector{Float64}()

gaus=Distributions.fit(Normal,eHits[100])
plot(gaus,linewidth=3)
for energy in keys(eHits)
    gausFn=Distributions.fit(Normal,eHits[energy]) #normalized by area
    println(energy," GeV: μ = ", gausFn.μ, "\tσ = ", gausFn.σ)
    push!(sigmasUncalib, gausFn.σ)
    push!(meansUncalib, gausFn.μ)
    plot!(gausFn,linewidth=3)
end
histogram!([eHits[energy]for energy in keys(eHits)], fillalpha=0.5, linewidth=0, label=map(string, keys(eHits)), xlabel="Uncalibrated Energy [GeV]",normalize=true)
xaxis!(:log10)

1 GeV: μ = 0.017162259017525185	σ = 0.003141314497495093
100 GeV: μ = 1.6888719539550865	σ = 0.044314385148727084
10 GeV: μ = 0.17033221812562682	σ = 0.011407095110428745
50 GeV: μ = 0.8475543339324533	σ = 0.028739756675865223
2 GeV: μ = 0.03404333950430362	σ = 0.0045045658644036835
5 GeV: μ = 0.08510260722238025	σ = 0.007635541322622888
20 GeV: μ = 0.33986118250905795	σ = 0.015707108943062625


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 10 
 
 
 0 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 Uncalibrated Energy [GeV] 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 576.228,369.666 576.45,369.663 576.673,369.659 576.894,369.653 577.116,369.646 577.336,369.636 577.556,369.623 577.776,369.607 577.995,369.585 578.214,369.558 
 578.432,369.524 578.649,369.48 578.867,369.425 579.083,369.357 579.299,369.273 579.515,369.171 579.73,369.045 579.945,368.894 580.159,368.713 580.373,368.497 
 580.586,368.243 580.798,367.944 581.011,367.597 581.222,367.197 581.434,366.738 581.644,366.218 581.855,365.631 582.065,364.976 582.274,364.25 582.483,363.453 
 582.691,362.585 582.899,361.649 583.107,360.648 583.314,359.588 583.521,358.477 583.727,357.325 583.933,356.143 584.138,354.944 584.343,353.743 584.547,352.557 
 584.751,351.402 584.954,350.295 585.158,349.256 585.36,348.301 585.562,347.447 585.764,346.71 585.965,346.102 586.166,345.636 586.367,345.32 586.567,345.16 
 586.766,345.16 586.966,345.32 587.164,345.636 587.363,346.102 587.561,346.71 587.758,347.447 587.955,348.301 588.152,349.256 588.348,350.295 588.544,351.402 
 588.74,352.557 588.935,353.743 589.129,354.944 589.324,356.143 589.518,357.325 589.711,358.477 589.904,359.588 590.097,360.648 590.289,361.649 590.481,362.585 
 590.672,363.453 590.864,364.25 591.054,364.976 591.245,365.631 591.435,366.218 591.624,366.738 591.813,367.197 592.002,367.597 592.191,367.944 592.379,368.243 
 592.566,368.497 592.754,368.713 592.941,368.894 593.127,369.045 593.313,369.171 593.499,369.273 593.685,369.357 593.87,369.425 594.054,369.48 594.239,369.524 
 594.423,369.558 594.607,369.585 594.79,369.607 594.973,369.623 595.155,369.636 595.338,369.646 595.519,369.653 595.701,369.659 595.882,369.663 596.063,369.666 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 30.6037,369.558 35.6639,369.514 40.4659,369.455 45.0349,369.377 49.3923,369.273 53.557,369.135 57.5452,368.956 61.3713,368.723 65.0479,368.423 68.5864,368.04 
 71.9966,367.552 75.2877,366.936 78.4675,366.165 81.5435,365.206 84.5221,364.022 87.4094,362.571 90.2108,360.805 92.9312,358.672 95.5752,356.115 98.147,353.073 
 100.65,349.48 103.089,345.27 105.466,340.374 107.784,334.725 110.047,328.257 112.257,320.913 114.416,312.64 116.526,303.397 118.59,293.158 120.61,281.912 
 122.588,269.669 124.525,256.458 126.422,242.337 128.283,227.388 130.107,211.717 131.897,195.462 133.653,178.785 135.377,161.873 137.07,144.935 138.733,128.198 
 140.367,111.904 141.973,96.301 143.553,81.6405 145.106,68.169 146.634,56.1217 148.138,45.7153 149.618,37.1421 151.075,30.5636 152.51,26.1058 153.924,23.855 
 155.316,23.855 156.688,26.1058 158.041,30.5636 159.374,37.1421 160.689,45.7153 161.986,56.1217 163.265,68.169 164.527,81.6405 165.772,96.301 167.001,111.904 
 168.214,128.198 169.411,144.935 170.594,161.873 171.762,178.785 172.916,195.462 174.055,211.717 175.181,227.388 176.294,242.337 177.394,256.458 178.481,269.669 
 179.555,281.912 180.618,293.158 181.668,303.397 182.707,312.64 183.735,320.913 184.752,328.257 185.757,334.725 186.753,340.374 187.737,345.27 188.712,349.48 
 189.676,353.073 190.631,356.115 191.576,358.672 192.512,360.805 193.439,362.571 194.356,364.022 195.265,365.206 196.165,366.165 197.056,366.936 197.939,367.552 
 198.814,368.04 199.681,368.423 200.54,368.723 201.391,368.956 202.235,369.135 203.071,369.273 203.899,369.377 204.721,369.455 205.535,369.514 206.342,369.558 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#3cb370; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 576.228,369.666 576.45,369.663 576.673,369.659 576.894,369.653 577.116,369.646 577.336,369.636 577.556,369.623 577.776,369.607 577.995,369.585 578.214,369.558 
 578.432,369.524 578.649,369.48 578.867,369.425 579.

In [6]:
# this function attempts a global fit and minimizes the offset b of the quadratic form y=axx+mx+b
# parameters are ecal energies (×2 for the hits in the outer layers), hcal energies, particle energy
function lineFitter(ecal, truValues)
    function model(x, p)
        energies = Dict{Int16, Float64}()
        for e in truValues
            calibrated=p[1].*x[e] + p[2].*x[e].^(3.0/2.0)
            n=Distributions.fit(Normal,calibrated)
            energies[e] = n.μ
        end
        # we are optimizing for the ratio of reconstructed energies to true values
        return [energies[e]/e for e in truValues]
    end
    fit = curve_fit(model, ecal, 1.0, [50.0,0.0])
    errors=estimate_errors(fit,0.95)
    return fit.param,errors
end
ECal, ECalErr = lineFitter(eHits, keys(eHits))
println("ECal calibration constant: ", ECal[1], " +/- ", ECalErr[1])
println("ECal fractional calibration constant: ", ECal[2], " +/- ", ECalErr[2])


ECal calibration constant: 58.464354462350954 +/- 0.20368142964059835
ECal fractional calibration constant: 0.5941123907726221 +/- 0.3032739547924162


In [7]:
sigmasCalib = Vector{Float64}()
meansCalib = Vector{Float64}()

gaus=Distributions.fit(Normal,ECal[2] .* eHits[100].^(3.0/2.0) + ECal[1] .* eHits[100])
plot(gaus,linewidth=3,label="")
for energy in keys(eHits)
    gausFn=Distributions.fit(Normal,ECal[2] .* eHits[energy].^ (3.0/2.0) + ECal[1] .* eHits[energy])
    println(energy," GeV: μ = ", gausFn.μ, "\tσ = ", gausFn.σ)
    push!(sigmasCalib,gausFn.σ)
    push!(meansCalib,gausFn.μ)
    plot!(gausFn,linewidth=3,label="")
end

histogram!([ECal[2] .* eHits[energy].^(3.0/2.0) + ECal[1] .* eHits[energy]  for energy in keys(eHits)], fillalpha=0.5, linewidth=0, label=map(string, keys(eHits)),normalize=true,xlabel="Calibrated Energy [GeV]")
xaxis!(:log10)

1 GeV: μ = 1.0047328518285341	σ = 0.18402665993282002
100 GeV: μ = 100.04311554554027	σ = 2.6398176900615793
10 GeV: μ = 10.00019946829975	σ = 0.6710361027249554
50 GeV: μ = 50.01549897078632	σ = 1.7028181659672075
2 GeV: μ = 1.994078214645652	σ = 0.2640977672574275
5 GeV: μ = 4.990263776481338	σ = 0.44836282738972316
20 GeV: μ = 19.987571535380546	σ = 0.9264044861526186


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 Calibrated Energy [GeV] 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 576.162,369.666 576.386,369.663 576.609,369.659 576.831,369.654 577.053,369.646 577.275,369.637 577.496,369.624 577.716,369.608 577.936,369.588 578.156,369.561 
 578.375,369.527 578.593,369.485 578.811,369.431 579.029,369.365 579.245,369.283 579.462,369.182 579.678,369.06 579.893,368.913 580.108,368.736 580.323,368.525 
 580.537,368.276 580.75,367.985 580.963,367.646 581.176,367.255 581.388,366.807 581.599,366.299 581.81,365.726 582.021,365.087 582.231,364.378 582.441,363.6 
 582.65,362.752 582.859,361.838 583.067,360.86 583.275,359.826 583.482,358.741 583.689,357.616 583.895,356.462 584.101,355.291 584.307,354.119 584.512,352.96 
 584.716,351.832 584.921,350.752 585.124,349.738 585.328,348.805 585.531,347.971 585.733,347.251 585.935,346.658 586.137,346.202 586.338,345.894 586.538,345.738 
 586.739,345.738 586.939,345.894 587.138,346.202 587.337,346.658 587.536,347.251 587.734,347.971 587.931,348.805 588.129,349.738 588.326,350.752 588.522,351.832 
 588.718,352.96 588.914,354.119 589.109,355.291 589.304,356.462 589.499,357.616 589.693,358.741 589.886,359.826 590.08,360.86 590.272,361.838 590.465,362.752 
 590.657,363.6 590.849,364.378 591.04,365.087 591.231,365.726 591.421,366.299 591.612,366.807 591.801,367.255 591.991,367.646 592.18,367.985 592.368,368.276 
 592.556,368.525 592.744,368.736 592.932,368.913 593.119,369.06 593.306,369.182 593.492,369.283 593.678,369.365 593.864,369.431 594.049,369.485 594.234,369.527 
 594.418,369.561 594.602,369.588 594.786,369.608 594.97,369.624 595.153,369.637 595.336,369.646 595.518,369.654 595.7,369.659 595.882,369.663 596.063,369.666 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 30.6037,369.559 35.6644,369.515 40.4663,369.457 45.0346,369.379 49.391,369.275 53.5543,369.139 57.5408,368.961 61.365,368.73 65.0396,368.432 68.5758,368.051 
 71.9836,367.567 75.2722,366.956 78.4495,366.19 81.5228,365.238 84.4987,364.062 87.3833,362.621 90.1818,360.868 92.8994,358.75 95.5406,356.211 98.1095,353.191 
 100.61,349.624 103.046,345.443 105.42,340.582 107.735,334.973 109.995,328.552 112.202,321.26 114.358,313.046 116.466,303.869 118.527,293.703 120.544,282.537 
 122.519,270.38 124.453,257.264 126.348,243.243 128.206,228.4 130.028,212.841 131.815,196.702 133.568,180.143 135.289,163.352 136.98,146.534 138.64,129.916 
 140.272,113.738 141.876,98.2459 143.453,83.6897 145.004,70.3141 146.529,58.3525 148.031,48.0202 149.508,39.508 150.963,32.9763 152.396,28.5502 153.807,26.3154 
 155.197,26.3154 156.567,28.5502 157.917,32.9763 159.248,39.508 160.561,48.0202 161.855,58.3525 163.132,70.3141 164.391,83.6897 165.634,98.2459 166.861,113.738 
 168.072,129.916 169.268,146.534 170.448,163.352 171.614,180.143 172.766,196.702 173.903,212.841 175.027,228.4 176.138,243.243 177.236,257.264 178.321,270.38 
 179.393,282.537 180.454,293.703 181.503,303.869 182.54,313.046 183.565,321.26 184.58,328.552 185.584,334.973 186.577,340.582 187.56,345.443 188.533,349.624 
 189.495,353.191 190.448,356.211 191.392,358.75 192.326,360.868 193.251,362.621 194.166,364.062 195.073,365.238 195.971,366.19 196.861,366.956 197.743,367.567 
 198.616,368.051 199.481,368.432 200.338,368.73 201.188,368.961 202.03,369.139 202.864,369.275 203.691,369.379 204.511,369.457 205.324,369.515 206.129,369.559 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#3cb370; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 576.162,369.666 576.386,369.663 576.609,369.659 576.831,369.654 577.053,369.646 577.275,369.637 577.496,369.624 577.716,369.608 577.936,369.588 578.156,369.561 
 578.375,369.527 578.593,369.485 578.811,369.

In [8]:
calibrated = Vector{Float64}()
x = Vector{Float64}()
xLin=[1.,2.,5.,10.,20.,50.,100.]
for e in keys(eHits)
    d = ECal[1] .* eHits[e] + ECal[2] .* eHits[e] .^ (3.0/2.0)
    gauss=Distributions.fit(Normal,d)
    push!(calibrated, gauss.μ)
    push!(x, 1.0*e)
end

line(x, p) = p[1] + p[2]*x
l = curve_fit(line, x, calibrated, [0.0, 1.0])
plot(x, calibrated, yerr=sigmasCalib, marker=stroke(2), line=false, label="data",xlabel="Initial Energy [GeV]",ylabel="Calibrated Energy [GeV]")
leg = @sprintf("y=%.2f + %.2f*x  ", l.param[1], l.param[2])
plot!(x, l.param[1]+l.param[2]*x, label=leg)
plot!(xLin,xLin,label="Diagonal")
xaxis!(:log10,(0.9,110.0))
yaxis!(:log10,(0.9,110.0))


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 Initial Energy [GeV] 
 
 
 Calibrated Energy [GeV] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 data 
 
 
 
 y=-0.01 + 1.00*x 
 
 
 
 Diagonal

In [9]:
calibratedLin = Vector{Float64}()
for e in keys(eHits)
    d = ECal[1] .* eHits[e] 
    gauss=Distributions.fit(Normal,d)
    push!(calibratedLin, gauss.μ)
end

l2 = curve_fit(line, x, calibratedLin, [0.0, 1.0])
plot(x, calibrated, yerr=sigmasCalib, marker=stroke(2), line=false, label="data",xlabel="Initial Energy [GeV]",ylabel="Linearly Calibrated Energy [GeV]")
leg2 = @sprintf("y=%.2f + %.2f*x  ", l2.param[1], l2.param[2])
plot!(x, l2.param[1]+l2.param[2]*x, label=leg2)
plot!(xLin,xLin,label="Diagonal")
xaxis!(:log10,(0.9,110.0))
yaxis!(:log10,(0.9,110.0))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 Initial Energy [GeV] 
 
 
 Linearly Calibrated Energy [GeV] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 data 
 
 
 
 y=0.06 + 0.99*x 
 
 
 
 Diagonal

In [10]:
linCalibRatio = Vector{Float64}()
linCalibRatioErr = Vector{Float64}()
for e in keys(eHits)
    d = (eHits[e]*ECal[1]) / (1.0*e)
    gauss=Distributions.fit(Normal,d)    
    push!(linCalibRatio, gauss.μ)
    push!(linCalibRatioErr, gauss.σ/sqrt(5000))
end

plot(x, linCalibRatio, yerr=linCalibRatioErr, marker=stroke(2), markersize=10, markercolor=:blue,line=false, label="Contained ",xlabel="Initial Energy [GeV]",ylabel="Energy Term / Initial Energy")
xaxis!(:log10,(0.9,110.0))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 0.990 
 
 
 0.995 
 
 
 1.000 
 
 
 1.005 
 
 
 Initial Energy [GeV] 
 
 
 Energy Term / Initial Energy 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Contained

In [11]:
calibRatio = Vector{Float64}()
calibRatioErr = Vector{Float64}()
for e in keys(eHits)
    d = (ECal[1] .* eHits[e] +ECal[2] .* eHits[e] .^(3.0/2.0))./ (1.0.*e)
    gauss=Distributions.fit(Normal,d)
    push!(calibRatio, gauss.μ)
    push!(calibRatioErr, gauss.σ/sqrt(5000))
end

plot(x, linCalibRatio, yerr=linCalibRatioErr, marker=stroke(2), markersize=10, markercolor=:blue,line=false, label="Contained")
plot!(x, calibRatio, yerr=calibRatioErr, marker=stroke(2),markersize=10, markercolor=:red,line=false,label="Corrected For Leakage Based on Shower Shape                 ",xlabel="Initial Energy [GeV]",ylabel="Energy Term / Initial Energy")

xaxis!(:log10,(0.9,110.0))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 0.990 
 
 
 0.995 
 
 
 1.000 
 
 
 1.005 
 
 
 Initial Energy [GeV] 
 
 
 Energy Term / Initial Energy 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Contained 
 
 
 
 
 
 
 
 Corrected For Leakage Based on Shower Shape

In [12]:
enRes = Vector{Float64}()
for index in 1:7
    push!(enRes,sigmasCalib[index]./meansCalib[index])
end

plot(x, abs((1.0-calibRatio)./enRes), yerr=(1-calibRatio)./enRes./sqrt(5000),marker=stroke(2), markersize=10, markercolor=:salmon,line=false, leg=false,xlabel="Initial Energy [GeV]",ylabel="(1-Calibrated Energy / Initial Energy)/Energy Resolution")
xaxis!(:log10,(0.9,110.0))
yaxis!((-0.001,0.03))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 0.00 
 
 
 0.01 
 
 
 0.02 
 
 
 0.03 
 
 
 Initial Energy [GeV] 
 
 
 (1-Calibrated Energy / Initial Energy)/Energy Resolution

In [13]:
plot(x, abs((1.0-linCalibRatio)./enRes), yerr=(1-linCalibRatio)./enRes./sqrt(5000),marker=stroke(2),markersize=10, line=false, label="Linear Calibration",xlabel="Initial Energy [GeV]",ylabel="(1-Energy / Initial Energy) / Energy Resolution")
plot!(x, abs((1.0-calibRatio)./enRes), yerr=(1-calibRatio)./enRes./sqrt(5000),marker=stroke(2), markersize=10,label="Full Nonlinear Calibration      ", line=false)

xaxis!(:log10,(0.9,110.0))
yaxis!((-0.02,0.65))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 Initial Energy [GeV] 
 
 
 (1-Energy / Initial Energy) / Energy Resolution 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Linear Calibration 
 
 
 
 
 
 
 
 Full Nonlinear Calibration

In [14]:
nonlinCalibRatio = Vector{Float64}()
nonlinCalibRatioErr = Vector{Float64}()
for e in keys(eHits)
    d = (ECal[2] .* eHits[e] .^(3.0/2.0))./ (1.0.*e)
    gauss=Distributions.fit(Normal,d)
    push!(nonlinCalibRatio, gauss.μ)
    push!(nonlinCalibRatioErr, gauss.σ./sqrt(5000.0))
end

plot(x, linCalibRatio, yerr=linCalibRatioErr, marker=stroke(2), markersize=10, markercolor=:yellow, line=false, label="Linear Calibration Term      ",xlabel="Initial Energy [GeV]",ylabel="Energy Term / Initial Energy")
plot!(x, nonlinCalibRatio, yerr=nonlinCalibRatioErr, marker=stroke(2), markersize=10, markercolor=:magenta, line=false,label="1/2 Calibration Term  ")

xaxis!(:log10,(0.9,110.0))
yaxis!(:log10,(0.0001,12))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 Initial Energy [GeV] 
 
 
 Energy Term / Initial Energy 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Linear Calibration Term 
 
 
 
 
 
 
 
 1/2 Calibration Term

In [15]:
histogram([eHitsBack[energy]./eHits[energy] for energy in [1,100,10]], bins=(0.0:0.001:0.03),fillalpha=0.5, linewidth=0, label=map(string, keys(eHits)),xlabel="Uncalibrated Deposits in Last 2 Layers [%]")

xaxis!((0.,0.022))
yaxis!((0.,5010.))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.000 
 
 
 0.005 
 
 
 0.010 
 
 
 0.015 
 
 
 0.020 
 
 
 0 
 
 
 1000 
 
 
 2000 
 
 
 3000 
 
 
 4000 
 
 
 5000 
 
 
 Uncalibrated Deposits in Last 2 Layers [%] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 100 
 
 
 
 
 10

In [16]:
back = Vector{Float64}()
for e in keys(eHits)
    push!(back, mean(eHitsBack[e],1)[1]./e.*100)
end

plot(x, nonlinCalibRatio, yerr=nonlinCalibRatioErr, marker=stroke(2),markersize=10, markercolor=:magenta, line=false,label="1/2 Term  ",xlabel="Initial Energy [GeV]",right_margin=30px)
plot!(x,back,marker=stroke(2),markersize=10, markercolor=:lightGreen,line=false,label="Uncalibrated Energy in Last 2 ECal Layers / Initial Energy*100                 ",ymirror=true,xlink=2, inset=(1,bbox(0,0,1,1)), bg_inside=RGBA(0,0,0,0))

xaxis!(:log10,(0.9,110.0),true)
yaxis!(:log10,(0.001,0.03),true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 Initial Energy [GeV] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1/2 Term 
 
 
 
 
 
 
 
 Uncalibrated Energy in Last 2 ECal Layers / Initial Energy*100 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2

In [17]:
hcalHitsRatio = Vector{Float64}()
for e in keys(eHits)
    push!(hcalHitsRatio, mean(hHits[e],1)[1]./e.*50.)
end

plot(x, nonlinCalibRatio, yerr=nonlinCalibRatioErr, marker=stroke(2), markersize=10, markercolor=:magenta,line=false,label="1/2 Term  ",xlabel="Initial Energy [GeV]",right_margin=30px)
plot!(x,hcalHitsRatio,marker=stroke(2),markersize=10, markercolor=:orange,line=false,label="Uncalibrated HCal Hits/Initial Energy*50          ",ymirror=true,xlink=2, inset=(1,bbox(0,0,1,1)), bg_inside=RGBA(0,0,0,0))

xaxis!(:log10,(0.9,110.0))
yaxis!(:log10,(0.001,0.03))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 Initial Energy [GeV] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1/2 Term 
 
 
 
 
 
 
 
 Uncalibrated HCal Hits/Initial Energy*50 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2

In [18]:
#from MIP vs scaled radiation length plot, linear fit extrapolation
fitIntegralL=[0.0009495162208586111, 0.001249982299277269, 0.0016381364639927664, 0.0023597250460691034, 0.0030983044984638354, 0.004435031937199862, 0.006134404577683722]
#from MIP vs radiation length plot, gamma fit extrapolation
fitIntegralG=[0.00021193102439427688, 0.000340690533861749, 0.0007747472067739187, 0.0009963353823439938, 0.0015979182805538118, 0.0024471932434884614, 0.003817725540069536]

plot(x, nonlinCalibRatio, yerr=nonlinCalibRatioErr, marker=stroke(2), markersize=10, markercolor=:magenta,line=false,label="1/2 Term  ",xlabel="Initial Energy [GeV]",right_margin=30px)
plot!(xLin,fitIntegralG,marker=stroke(2),markersize=10, markercolor=:green,line=false,label="Integral value of Gamma Fit Leakage Extrapolation              ",ymirror=true,xlink=2, inset=(1,bbox(0,0,1,1)), bg_inside=RGBA(0,0,0,0))
plot!(xLin,fitIntegralL,marker=stroke(2),markersize=10, markercolor=:lightBlue,line=false,label="Integral value of Linear Fit Leakage Extrapolation")

xaxis!(:log10,(0.9,110.0))
yaxis!(:log10,(0.0001,0.05))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 Initial Energy [GeV] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1/2 Term 
 
 
 
 
 
 
 
 Integral value of Gamma Fit Leakage Extrapolation 
 
 
 
 
 
 
 
 Integral value of Linear Fit Leakage Extrapolation 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2

In [19]:
plot(x, nonlinCalibRatio, yerr=nonlinCalibRatioErr, marker=stroke(2),markersize=10, markercolor=:magenta, line=false,label="1/2 Term  ",xlabel="Initial Energy [GeV]",right_margin=30px)
plot!(x,back,marker=stroke(2),markersize=10, markercolor=:lightGreen,line=false,label="Uncalibrated Energy in Last 2 Layers / Initial Energy*100               ")
plot!(x,hcalHitsRatio,marker=stroke(2),markersize=10, markercolor=:orange,line=false,label="Uncalibrated HCal Hits/Initial Energy*50          ")
plot!(xLin,fitIntegralG,marker=stroke(2),markersize=10, markercolor=:green,line=false,label="Integral value of Gamma Fit Leakage Extrapolation")
plot!(xLin,fitIntegralL,marker=stroke(2),markersize=10, markercolor=:lightBlue,line=false,label="Integral value of Linear Fit Leakage Extrapolation  ",ymirror=true,xlink=2, inset=(1,bbox(0,0,1,1)), bg_inside=RGBA(0,0,0,0))

xaxis!(:log10,(0.9,110.0))
yaxis!(:log10,(0.00008,0.1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 Initial Energy [GeV] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1/2 Term 
 
 
 
 
 
 
 
 Uncalibrated Energy in Last 2 Layers / Initial Energy*100 
 
 
 
 
 
 
 
 Uncalibrated HCal Hits/Initial Energy*50 
 
 
 
 
 
 
 
 Integral value of Gamma Fit Leakage Extrapolation 
 
 
 
 
 
 
 
 Integral value of Linear Fit Leakage Extrapolation 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1

In [20]:
hcalHitsScaled = Vector{Float64}()
nonlinCalibRatioScaled = Vector{Float64}()
nonlinCalibRatioScaledErr = Vector{Float64}()
integralLScaled = Vector{Float64}()
integralGScaled = Vector{Float64}()
backScaled = Vector{Float64}()
for e in keys(eHits)
    d = (ECal[2] .* eHits[e] .^(3.0/2.0))./ (1.0.*e) ./ sqrt(1.0.*e)
    gauss=Distributions.fit(Normal,d)
    push!(nonlinCalibRatioScaled, gauss.μ)
    push!(nonlinCalibRatioScaledErr, gauss.σ./sqrt(5000.0))
end
for e in keys(eHits)
    push!(hcalHitsScaled, mean(hHits[e],1)[1]./e/sqrt(1.0.*e))
end

for index in 1:7
    push!(integralGScaled, fitIntegralG[index]./sqrt(xLin[index]))
    push!(integralLScaled, fitIntegralL[index]./sqrt(xLin[index]))
    push!(backScaled, back[index]./sqrt(x[index]))
end

plot(x, nonlinCalibRatioScaled, yerr=nonlinCalibRatioScaledErr, marker=stroke(2), markersize=10, markercolor=:darkMagenta,line=false,label="1/2 Term/sqrt(E)  ")
plot!(x,hcalHitsScaled,marker=stroke(2),markersize=10, markercolor=:brown,line=false,label="Uncalibrated HCal Hits/Initial Energy/sqrt(E)          ")
plot!(x,backScaled,marker=stroke(2),markersize=10, markercolor=:darkCyan,line=false,label="Uncalibrated Energy in Last 2 Layers / Initial Energy *100 /sqrt(E)                  ")
plot!(xLin,integralGScaled,marker=stroke(2),markersize=10, markercolor=:darkGreen,line=false,label="Integral value of Gamma Fit Leakage Extrapolation/sqrt(E)                  ")
plot!(xLin,integralLScaled,marker=stroke(2),markersize=10, markercolor=:darkBlue,line=false,label="Integral value of Linear Fit Leakage Extrapolation/sqrt(E)                  ",xlabel="Initial Energy [GeV]",right_margin=30px,ymirror=true,xlink=2, inset=(1,bbox(0,0,1,1)), bg_inside=RGBA(0,0,0,0))

xaxis!(:log10,(0.9,110.0))
yaxis!(:log10,(0.00001,0.05))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 Initial Energy [GeV] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1/2 Term/sqrt(E) 
 
 
 
 
 
 
 
 Uncalibrated HCal Hits/Initial Energy/sqrt(E) 
 
 
 
 
 
 
 
 Uncalibrated Energy in Last 2 Layers / Initial Energy *100 /sqrt(E) 
 
 
 
 
 
 
 
 Integral value of Gamma Fit Leakage Extrapolation/sqrt(E) 
 
 
 
 
 
 
 
 Integral value of Linear Fit Leakage Extrapolation/sqrt(E) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2

In [21]:
plot(xLin,fitIntegralG,marker=stroke(2),markersize=10, markercolor=:green,line=false,label="Integral value of Gamma Fit Leakage Extrapolation                  ")
plot!(xLin,fitIntegralL,marker=stroke(2),markersize=10, markercolor=:lightBlue,line=false,label="Integral value of Linear Fit Leakage Extrapolation                  ",xlabel="Initial Energy [GeV]",right_margin=30px,ymirror=true,xlink=2, inset=(1,bbox(0,0,1,1)), bg_inside=RGBA(0,0,0,0))
plot!(x,hcalHitsRatio,marker=stroke(2),markersize=10, markercolor=:orange,line=false,label="Uncalibrated HCal Hits/Initial Energy*50          ")

xaxis!(:log10,(0.9,110.0))
yaxis!(:log10,(0.0001,0.05))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 Initial Energy [GeV] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Integral value of Gamma Fit Leakage Extrapolation 
 
 
 
 
 
 
 
 Integral value of Linear Fit Leakage Extrapolation 
 
 
 
 
 
 
 
 Uncalibrated HCal Hits/Initial Energy*50 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2